In [ ]:
!pip install azure-ai-documentintelligence -q
!pip install azure-ai-formrecognizer -q
!pip install azure -q
!pip install PyPDF2 -q
!pip install tabula-py -q
!pip install openai -q
!pip install textwrap -q
!pip install pdf2image pillow -q
!pip install pdf2image -q
!sudo apt-get install poppler-utils -q
!pip install pytesseract -q
!pip install fitz frontend PyMuPDF -q

In [ ]:
from PyPDF2 import PdfReader
import pandas as pd
import numpy as np
import re
import tabula
from openai import OpenAI
import os
import time
import textwrap
import io
import base64
import csv
from pdf2image import convert_from_path
from openai import OpenAI
import PyPDF2
import tempfile
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

HEADERS - Vision

In [ ]:
class PDFExtractor:
    def __init__(self, pdf_path, output_csv_path):
        self.pdf_path = pdf_path
        self.output_csv_path = output_csv_path
        self.column_headers = []

    async def process_pdf(self):
        pages_to_extract, total_pages = self.extract_pages_from_pdf()
        images = self.convert_pdf_to_images(pages_to_extract)

        for i, image in enumerate(images):
            column_headers = await self.extract_column_headers(image, pages_to_extract[i])
            self.column_headers.extend(column_headers)

        self.column_headers = list(set(self.column_headers))

        pdf_text = self.extract_text_from_pdf(pages_to_extract)
        csv_content = await self.extract_tabular_data(pdf_text)

        self.save_to_csv(csv_content)

    def extract_pages_from_pdf(self):
        reader = PdfReader(self.pdf_path)
        total_pages = len(reader.pages)

        if total_pages >= 6:
            pages_to_extract = list(range(4)) + list(range(total_pages - 2, total_pages))
        else:
            pages_to_extract = list(range(total_pages))

        return pages_to_extract, total_pages

    def convert_pdf_to_images(self, pages_to_extract):
        return convert_from_path(self.pdf_path, first_page=min(pages_to_extract) + 1, last_page=max(pages_to_extract) + 1)

    async def extract_column_headers(self, image, page_number):
        json_format = """
        {
            "column_headers": ["<header1>", "<header2>", ...],
            "column_values": ["<row1 value>", "<row2 value>", ...]
        }
        """
        vision_prompt = f"""
        Extract only the column headers and a single row of column values from the table in this image.
        If there's no table or no clear column headers, return an empty list.
        Return the result in the following JSON format:
        {json_format}
        """

        extractor = LLMImageExtractor(vision_prompt, image, 'openai')
        parsed_json = await extractor.run()

        print(f"Extracted from page {page_number + 1}: Headers: {parsed_json['column_headers']}, Values: {parsed_json['column_values']}")
        return parsed_json['column_headers']

    def extract_text_from_pdf(self, pages_to_extract):
        with open(self.pdf_path, 'rb') as file:
            reader = PdfReader(file)
            text = ""
            for page_num in pages_to_extract:
                text += reader.pages[page_num].extract_text() + "\n"
        return text

    async def extract_tabular_data(self, pdf_text):
        json_format = """
        {
            "csv_content": "header1,header2,header3\\nvalue1,value2,value3\\n..."
        }
        """
        prompt = f"""
        Using these column headers: {', '.join(self.column_headers)}
        Extract all tabular data from the following text and format it as CSV.
        Include the header row. Return the result in the following JSON format:
        {json_format}

        Text from PDF:
        {pdf_text}
        """

        client = OpenAI(api_key="")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            max_tokens=4000,
        )

        parsed_json = json.loads(response.choices[0].message.content)
        return parsed_json['csv_content']

    def save_to_csv(self, csv_content):
        with open(self.output_csv_path, 'w', newline='', encoding='utf-8') as file:
            file.write(csv_content)
        print(f"Saved CSV to {self.output_csv_path}")

In [ ]:
class LLMImageExtractor:
    def __init__(self, prompt, image, service='openai'):
        self.prompt = prompt
        self.image = image
        self.service = service

    async def run(self):
        if self.service == 'openai':
            return await self.callOpenAI(self.prompt, self.image)
        else:
            raise ValueError(f"Invalid service: {self.service}. Please choose 'openai'.")

    async def callOpenAI(self, prompt, image):
        client = OpenAI(api_key="")

        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        image_base64 = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}}
                    ]
                }
            ],
            max_tokens=300,
            response_format={"type": "json_object"}
        )

        return json.loads(response.choices[0].message.content)

In [ ]:
async def main():
    pdf_path = "/content/spoils3.pdf"
    output_csv_path = "/content/op1.csv"

    extractor = PDFExtractor(pdf_path, output_csv_path)
    await extractor.process_pdf()

# if __name__ == "__main__":
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(main())

In [ ]:
await main()

AZURE

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = ""
key = ""

formUrl = ""

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)
result = poller.result()

# Process tables
all_tables_data = []

print(result.tables[0])

for table_idx, table in enumerate(result.tables):
    print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")

    table_data = [[''] * table.column_count for _ in range(table.row_count)]

    for cell in table.cells:
        row, col = cell.row_index, cell.column_index
        content = cell.content.strip()
        table_data[row][col] = content

    all_tables_data.append(table_data)

    print("Table data:")
    for row in table_data:
        print(row)
    print()

print("----------------------------------------")

# Now all_tables_data contains the table data in the desired format
# You can access individual tables like this:
# first_table = all_tables_data[0]
# second_table = all_tables_data[1]
# etc.

DocumentTable(row_count=2, column_count=2, cells=[DocumentTableCell(kind=columnHeader, row_index=0, column_index=0, row_span=1, column_span=1, content=Invoice #, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=1.1697, y=3.166), Point(x=2.7405, y=3.1612), Point(x=2.7452, y=3.4334), Point(x=1.1697, y=3.4334)])], spans=[DocumentSpan(offset=46, length=9)]), DocumentTableCell(kind=columnHeader, row_index=0, column_index=1, row_span=1, column_span=1, content=TN31556318, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=2.7405, y=3.1612), Point(x=4.4926, y=3.1612), Point(x=4.4974, y=3.4334), Point(x=2.7452, y=3.4334)])], spans=[DocumentSpan(offset=56, length=10)]), DocumentTableCell(kind=content, row_index=1, column_index=0, row_span=1, column_span=1, content=Special Payee, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=1.1697, y=3.4334), Point(x=2.7452, y=3.4334), Point(x=2.7548, y=3.7104), Point(x=1.1697, y=3.7104)])], spans=[DocumentSpan(of

GPT

In [ ]:
table_1_headers = all_tables_data[1][0]
print("Table 1 Headers:", table_1_headers)

Table 1 Headers: ['Item #', 'UPC', 'Brand', 'Description', 'Master Ref #', 'Qty', 'Date', 'Range', 'PO #', 'Scan $']


In [ ]:
import openai

openai.api_key = ""

prompt = f"""
Analyze the following table headers and provide a brief explanation of what each column likely represents:

{', '.join(table_1_headers)}

Provide your response in JSON format with the header as the key and the explanation as the value.
"""
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that explains table headers."},
        {"role": "user", "content": prompt}
    ],
    response_format={"type": "json_object"},
)

header_explanations = response.choices[0].message.content
print("Header Explanations:", header_explanations)

Header Explanations: {
  "Item #": "A unique identifier assigned to each product or item in the inventory.",
  "UPC": "The Universal Product Code, a barcode used to identify the product.",
  "Brand": "The name of the company or maker of the product.",
  "Description": "A brief summary or details about the product.",
  "Master Ref #": "A master reference number, possibly used for internal tracking or linking related records.",
  "Qty": "The quantity of the item available or reported.",
  "Date": "The specific date associated with the record, which could be the date of inventory, purchase, or entry.",
  "Range": "A categorical or numeric range that might indicate product category, price, size, or another characteristic.",
  "PO #": "The Purchase Order number, a unique identifier for the purchase transaction.",
  "Scan $": "The scanned price or cost of the item, likely obtained through a sales or inventory scanning process."
}


In [ ]:
table_2_data = all_tables_data[2]
table_2_sample = table_2_data[:3]

prompt = f"""
Given the following headers and their explanations:
{header_explanations}

And the following sample data from another table:
{table_2_sample}

Assign the most appropriate headers from the first table to each column of the second table. Use only the available headers and use one header only once.

Provide your response in JSON format with the column index as the key and the assigned header as the value.
"""

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that assigns headers to table columns."},
        {"role": "user", "content": prompt}
    ],
    response_format={"type": "json_object"},
)

assigned_headers = response.choices[0].message.content
print("Assigned Headers for Table 2:", assigned_headers)

assigned_headers_json = json.loads(assigned_headers)

column_names = []

for i in range(len(assigned_headers_json)):
    column_names.append(assigned_headers_json[str(i)])

Assigned Headers for Table 2: {
  "0": "Item #",
  "1": "UPC",
  "2": "Brand",
  "3": "Description",
  "4": "Master Ref #",
  "5": "Date",
  "6": "Range",
  "7": "Scan $"
}


In [ ]:
column_names

['Item #',
 'UPC',
 'Brand',
 'Description',
 'Master Ref #',
 'Date',
 'Range',
 'Scan $']

In [ ]:
table_2_df = pd.DataFrame(all_tables_data[2], columns=column_names)
table_2_df

,Item #,UPC,Brand,Description,Master Ref #,Date,Range,Scan $
0,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,7/2/2023,7/29/2023,$86.68
1,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,7/2/2023,7/29/2023,$14.82
2,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,1/30/2022,1/28/2023,"$4,413.46"
3,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,4/30/2023,7/29/2023,$4.14
4,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,1/30/2022,1/28/2023,"$4,320.10"
5,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,1/30/2022,1/28/2023,$840.20
6,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,1/30/2022,1/28/2023,$746.84
7,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,4/30/2023,7/29/2023,$19.72
8,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,1/30/2022,4/30/2022,"$2,317.25"
9,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,1/30/2022,4/30/2022,"$2,317.26"


In [ ]:
table_1_df = pd.DataFrame(all_tables_data[1][1:], columns=all_tables_data[1][0])
table_1_df

,Item #,UPC,Brand,Description,Master Ref #,Qty,Date,Range,PO #,Scan $
0,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,7/2/2023,7/29/2023,,$16.89
1,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,6/4/2023,7/29/2023,,$277.97
2,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,4/30/2023,7/29/2023,,$23.10
3,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,4/2/2023,4/29/2023,,$163.31
4,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,7/2/2023,7/29/2023,,$88.75
5,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,4/2/2023,4/29/2023,,$145.73
6,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,6/4/2023,7/29/2023,,$46.45
7,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,6/4/2023,7/29/2023,,$47.23


In [ ]:
final_df = pd.concat([table_1_df, table_2_df], axis=0, ignore_index=True)
final_df

,Item #,UPC,Brand,Description,Master Ref #,Qty,Date,Range,PO #,Scan $
0,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,7/2/2023,7/29/2023,,$16.89
1,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,6/4/2023,7/29/2023,,$277.97
2,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,4/30/2023,7/29/2023,,$23.10
3,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,4/2/2023,4/29/2023,,$163.31
4,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,7/2/2023,7/29/2023,,$88.75
5,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,4/2/2023,4/29/2023,,$145.73
6,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,,6/4/2023,7/29/2023,,$46.45
7,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,,6/4/2023,7/29/2023,,$47.23
8,00346001,855258006249,NORA SNACKS,TEMPURA SPICY,12566010924,NaN,7/2/2023,7/29/2023,NaN,$86.68
9,00345748,855258006225,NORA SNACKS,SEAWEED CRISPY ORIGINA,12566010924,NaN,7/2/2023,7/29/2023,NaN,$14.82


### TRIAL

In [ ]:
all_headers = [['Invoice #', 'Special Payee'], ['Item #', 'UPC', 'Brand', 'Description', 'Master Ref #', 'Qty', 'Date Range', 'PO #', 'Scan $'], ['SKU', 'UPC', 'Description', 'Category', 'Item Number', 'Start Date', 'End Date', 'Total Sales']]

In [ ]:
def get_longest_header(all_headers):
    return max(all_headers, key=len)

# After extracting headers using LLMExtractor
longest_header = get_longest_header(extracted_headers)
longest_header

['Item #',
 'UPC',
 'Brand',
 'Description',
 'Master Ref #',
 'Qty',
 'Date Range',
 'PO #',
 'Scan $']

In [ ]:
def get_table_with_max_columns(all_tables_data):
    return max(all_tables_data, key=lambda table: len(table[0]) if table else 0)

def assign_headers(table, headers):
    if not table[0] or all(cell == '' for cell in table[0]):
        table[0] = headers[:len(table[0])]
    return table

max_column_table = get_table_with_max_columns(all_tables_data)
max_column_table = assign_headers(max_column_table, longest_header)

In [ ]:
def make_headers_unique(headers):
    seen = {}
    unique_headers = []
    for header in headers:
        if header in seen:
            seen[header] += 1
            unique_headers.append(f"{header}_{seen[header]}")
        else:
            seen[header] = 0
            unique_headers.append(header)
    return unique_headers

def assign_column_names(table, available_headers):
    prompt = f"""
    Given the following headers:
    {', '.join(available_headers)}

    And the following sample data from a table:
    {table[:3]}

    Assign the most appropriate headers from the available headers to each column of this table.
    Use only the exact headers provided, and use each header only once.
    If a column doesn't match any header exactly, leave it as an empty string.

    Provide your response in JSON format with the column index as the key and the assigned header as the value.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that assigns headers to table columns."},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
    )

    assigned_headers = json.loads(response.choices[0].message.content)
    headers = [assigned_headers.get(str(i), f'Column_{i}') for i in range(len(table[0]))]
    return make_headers_unique(headers)

def combine_tables(all_tables_data, available_headers):
    combined_df = pd.DataFrame()

    for table in all_tables_data:
        if table and len(table) > 1:
            assigned_headers = assign_column_names(table, available_headers)
            df = pd.DataFrame(table[1:], columns=assigned_headers)


            df.columns = make_headers_unique(df.columns.tolist())

            combined_df = pd.concat([combined_df, df], axis=0, ignore_index=True)

    return combined_df

In [ ]:
async def main():
    pdf_path = "/content/spoils3.pdf"
    output_csv_path = "/content/op2.csv"

    # extractor = PDFExtractor(pdf_path, output_csv_path)
    # all_headers, all_tables_data = await extractor.process_pdf()
    # all_headers = [['Invoice #', 'Special Payee'], ['Item #', 'UPC', 'Brand', 'Description', 'Master Ref #', 'Qty', 'Date Range', 'PO #', 'Scan $'], ['SKU', 'UPC', 'Description', 'Category', 'Item Number', 'Start Date', 'End Date', 'Total Sales']]
    all_headers = ['UPC', 'ITEM', 'BRAND', 'Cust Name', 'Date', 'Inv #', 'Qty', 'Inv %', 'Amt']
    longest_header = get_longest_header(all_headers)

    final_df = combine_tables(all_tables_data, longest_header)

    final_df.to_csv(output_csv_path, index=False)
    print(f"Saved CSV to {output_csv_path}")


await main()

Saved CSV to /content/op2.csv


In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = ""
key = ""

formUrl = ""

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)
result = poller.result()

# Process tables
all_tables_data = []

for table_idx, table in enumerate(result.tables):
    print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")

    table_data = [[''] * table.column_count for _ in range(table.row_count)]

    for cell in table.cells:
        row, col = cell.row_index, cell.column_index
        content = cell.content.strip()
        table_data[row][col] = content

    all_tables_data.append(table_data)

    print("Table data:")
    for row in table_data:
        print(row)
    print()

print("----------------------------------------")

# Now all_tables_data contains the table data in the desired format
# You can access individual tables like this:
# first_table = all_tables_data[0]
# second_table = all_tables_data[1]
# etc.

HttpResponseError: (InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Inner error: {
    "code": "InvalidContent",
    "message": "The file is corrupted or format is unsupported. Refer to documentation for the list of supported formats."
}